### Access rules for descriptor attribute 'd' when called as x.d
(ref: https://docs.python.org/3/howto/descriptor.html) 

if (x is an object/instance):
    call object.__getattribute__() (unless overwritten) and that function does the following:
        try x.d transformed to type(x).__dict__['d'].__get__(x,type(x) (if d is a 'data' decscriptor)
        else try x.__dict__['d'] (standard instance variable lookup which tries class and base classes chain as well) 
        else try type(x).__dict__['d'].__get__(x,type(x) (if d is a 'non-data' decscriptor)
        else raise AttributeError and try x.__getattr__(d) if present (follows up the class and base class chain)
elseif (x is a class):
    call type.__getattribute__() which attempts the following:
        try x.__dict__['d'].__get__(None,x) (follow base classes up the chain)
        else try x.__dict__['d'] (follow base classes up the chain)

for super(classB, obj).m():
    if class A immediately follows B in B.__mro__:
        try A.__dict__['m'].__get__(obj, B) (if 'm' is a descriptor; 'm' in this instance is a function)
        else try A.__dict__['m'] (when m is not a descriptor)
        else try above rules for object.__getattribute__ to locate obj.m
        

In [2]:
### aside ...
class D(object):
    def f(self,x):
        return x
d = D()

In [16]:

print(D.__dict__['f'])
print(id(D.__dict__['f']))
print(D.f)
print(id(D.f))
print(d.f)
print(id(d.f))
d
print(id(d))
print(f"id of d.f.__func__ {id(d.f.__func__)}")
print(id(d.f.__self__))
print(id(d.f.__class__))
print(d.f.__class__)
print(id(D))
print(id(d))

<function D.f at 0x104f819d8>
4378335704
<function D.f at 0x104f819d8>
4378335704
<bound method D.f of <__main__.D object at 0x104f829e8>>
4377652040
4378339816
id of d.f.__func__ 4378335704
4378339816
4331865136
<class 'method'>
140426961062968
4378339816


In [60]:
# python version
import sys
print(sys.version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 12:04:33) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [29]:
class DD:
    def __init__(self,name):
        self._name = name
    
    def __get__(self,obj,objclass = None):
        print(f"inside get: {self._name } -- obj: {repr(obj)} -- objclass {repr(objclass)}")
        return self._name
    def __set__(self,obj,value):
        self._name = value
        obj[self] = value + value
        print(f"inside set: {self._name } -- obj: {repr(obj)} -- value {value}")

In [30]:
class TestClass:
    dd = DD('bob')

In [31]:
dd1 = DD('pop')
obj1 = {}
dd1.__get__(obj1)
dd1.__set__(obj1,'sage')

inside get: pop -- obj: {} -- objclass None
inside set: sage -- obj: {<__main__.DD object at 0x10cc49ef0>: 'sagesage'} -- value sage


In [44]:
type(obj1)
obj1.values()
obj1.keys()

dict_keys([<__main__.DD object at 0x10cc49ef0>])

In [24]:
class Parent:
        parent1 = DD("parentCori")

class Child(Parent):
        child1 = DD("childBasil")

In [25]:
bas = Child()
bas.child1

inside get: childBasil -- obj: <__main__.Child object at 0x1065daeb8> -- objclass <class '__main__.Child'>


'childBasil'

In [26]:
bas.parent1

inside get: parentCori -- obj: <__main__.Child object at 0x1065daeb8> -- objclass <class '__main__.Child'>


'parentCori'

In [27]:
Child.child1

inside get: childBasil -- obj: None -- objclass <class '__main__.Child'>


'childBasil'

In [28]:
Child.parent1

inside get: parentCori -- obj: None -- objclass <class '__main__.Child'>


'parentCori'

In [29]:
Child.__dict__

mappingproxy({'__doc__': None,
              '__module__': '__main__',
              'child1': <__main__.DD at 0x1065dab70>})

In [30]:
Parent.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Parent' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Parent' objects>,
              'parent1': <__main__.DD at 0x1065dab38>})

In [31]:
DD.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'DD' objects>,
              '__doc__': None,
              '__get__': <function __main__.DD.__get__>,
              '__init__': <function __main__.DD.__init__>,
              '__module__': '__main__',
              '__set__': <function __main__.DD.__set__>,
              '__weakref__': <attribute '__weakref__' of 'DD' objects>})

In [32]:
bas.__class__.__dict__

mappingproxy({'__doc__': None,
              '__module__': '__main__',
              'child1': <__main__.DD at 0x1065dab70>})

In [33]:
bas.child1 = "Pepper"

inside set: Pepper -- obj: <__main__.Child object at 0x1065daeb8> -- value Pepper


In [34]:
bas.__dict__

{}

In [35]:
bas.child1

inside get: Pepper -- obj: <__main__.Child object at 0x1065daeb8> -- objclass <class '__main__.Child'>


'Pepper'

In [36]:
type(bas.child1)

inside get: Pepper -- obj: <__main__.Child object at 0x1065daeb8> -- objclass <class '__main__.Child'>


str

In [37]:
bas.__class__.child1 = "helloWorld"

In [38]:
bas.child1

'helloWorld'

In [39]:
bas.__class__.__dict__

mappingproxy({'__doc__': None,
              '__module__': '__main__',
              'child1': 'helloWorld'})

In [40]:
bas.__class__.child1

'helloWorld'

In [42]:
bas.__class__.__class__

type

In [43]:
Child.__class__

type

In [44]:
bas.__class__

__main__.Child

In [45]:
type.__class__

type

In [46]:
bas.__bases__

AttributeError: 'Child' object has no attribute '__bases__'

In [47]:
bas.__class__.__bases__

(__main__.Parent,)

In [48]:
Parent.__bases__

(object,)

In [49]:
type.__bases__

(object,)

In [50]:
Parent.parent1

inside get: parentCori -- obj: None -- objclass <class '__main__.Parent'>


'parentCori'

In [51]:
Parent.__class__

type

In [52]:
Parent.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'Parent' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'Parent' objects>,
              'parent1': <__main__.DD at 0x1065dab38>})

In [53]:
Child.parent1

inside get: parentCori -- obj: None -- objclass <class '__main__.Child'>


'parentCori'

In [54]:
type(Child)

type

In [55]:
type(bas)

__main__.Child

In [57]:
bas.__class__

__main__.Child